# Datan analysointi S2020 - kerta 2
<a href="#1">read_csv</a>  
<a href="#2">Muita lukufunktioita</a>  
<a href="#3">DataFramen kirjoittaminen tiedostoon</a>  
<a href="#4">Datan siivoaminen</a>  
<a href="#5">Puuttuvien arvojen käsittely</a>  
<a href="#6">Duplikaattien poistaminen</a>  
<a href="#8">Kategorisointi</a>  
<a href="#9">NumPy-funktiot DataFramelle</a>  
<a href="#10">apply, applymap, map</a>  
<a href="#11">pandasin merkkijonofunktiot</a>  
 

## Tiedon lukeminen DataFrameen
pandas tarjoaa monia funktioita datan lukemiseen eri formaateista, mm.
* read_csv	 
* read_table	
* read_clipboard	
* read_excel	
* read_html	
* read_json	
* read_sql

Nämä funktiot tarjoavat paljon valinnisia parametreja, joilla voidaan 
* määrittää DataFramen indeksit (rivi- ja sarakeotsikot)
* määrittää datalle muunnoksia
* parsia päivämääriä esim. eri sarakkeissa olevien arvojen perusteella
* määrittää miten puuttuvat arvot tai epäpuhdas data käsitellään

<a id="1"> </a>
### read_csv
`read_csv` lukee tiedon tekstitiedostosta, jossa tietueet on eroteltu välimerkein. Oletusvälimerkki on pilkku.

`read_csv`-funktiolla on n. 50 mahdollista parametria [speksi](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), mutta ainoa välttämätön on tieodston nimi, polku tai URL.

Esimerkin tiedosto sisältää rivit:  
`a,b,c,d,viesti
1,2,3,4,hei
5,6,7,8,maailma
9,10,11,12,taas`


In [5]:
import pandas as pd

df = pd.read_csv('https://bit.ly/2S0LcUJ')

df

,a,b,c,d,viesti
0,1,2,3,4,hei
1,5,6,7,8,maailma
2,9,10,11,12,taas


Nähdään, että pandas loi indeksin (riviotsikot) 0,1,2 jne ja otti tiedoston 1. rivin sarakeotsikoiksi.

Jos tiedostossa ei ole otsikkoriviä, voidaan joko määrittää `header=None`, jolloin pandas luo sarakeotsikot 0,1,2... tai määrittää sarakeotsikot itse `names`-parametrilla.

Esimerkin tiedosto sisältää rivit:  
`1,2,3,4,hei
5,6,7,8,maailma
9,10,11,12,taas`

In [6]:
df = pd.read_csv('https://bit.ly/2FJe8uJ', header=None)

df

,0,1,2,3,4
0,1,2,3,4,hei
1,5,6,7,8,maailma
2,9,10,11,12,taas


In [7]:
df = pd.read_csv('https://bit.ly/2FJe8uJ', names=['eka', 'toka', 'kolmas', 'vika', 'viesti'])

df

,eka,toka,kolmas,vika,viesti
0,1,2,3,4,hei
1,5,6,7,8,maailma
2,9,10,11,12,taas


Riviotsikoiksi eli taulukon indeksiksi voidaan ottaa joku datan sarakkeista `index_col`-parametrilla:

In [8]:
df = pd.read_csv('https://bit.ly/2FJe8uJ', index_col='viesti', names=['eka', 'toka', 'kolmas', 'vika', 'viesti'])

df

,eka,toka,kolmas,vika
viesti,,,,
hei,1,2,3,4
maailma,5,6,7,8
taas,9,10,11,12


In [9]:
df = pd.read_csv('https://bit.ly/2S0LcUJ', index_col='viesti') #tässä tiedostossa 1. rivi antoi sarakeotsikot

df

,a,b,c,d
viesti,,,,
hei,1,2,3,4
maailma,5,6,7,8
taas,9,10,11,12


Muilla parametreilla voidaan määritää mm. tietueiden välimerkki (oletuksena pilkku) tai desimaalierotin (oletuksena piste).

Esimerkkitiedosto:  
`Vuosi;Kk;Pv;Klo;Aikavyöhyke;Sademäärä;Lumensyvyys;Ylin;Alin
1959;1;1;00:00;UTC;1;34;-3,6;-9,8
1959;1;2;00:00;UTC;1,5;35;-4;-8
1959;1;3;00:00;UTC;10,6;36;0;-8,7
1959;1;4;00:00;UTC;1,7;38;0,7;-1,1
1959;1;5;00:00;UTC;7,1;40;-0,7;-2,8
1959;1;6;00:00;UTC;2,3;50;-1,2;-4,8`

In [10]:
df = pd.read_csv('https://bit.ly/2CFermr', sep=';', decimal=',')

df

,Vuosi,Kk,Pv,Klo,Aikavyöhyke,Sademäärä,Lumensyvyys,Ylin,Alin
0,1959,1,1,00:00,UTC,1.0,34,-3.6,-9.8
1,1959,2,1,00:00,UTC,-1.0,63,-6.1,-13.1
2,1959,3,1,00:00,UTC,-1.0,55,8.2,-1.5
3,1959,4,1,00:00,UTC,7.8,27,5.0,-10.5
4,1959,5,1,00:00,UTC,-1.0,-1,17.9,-0.9
5,1959,6,1,00:00,UTC,3.6,-1,14.0,1.2


`usecols`-parametrilla voidaan ottaa vain halutut sarakkeet:

In [11]:
df = pd.read_csv('https://bit.ly/2CFermr', sep=';', decimal=',' , usecols=['Vuosi', 'Kk', 'Pv', 'Lumensyvyys'])

df

,Vuosi,Kk,Pv,Lumensyvyys
0,1959,1,1,34
1,1959,2,1,63
2,1959,3,1,55
3,1959,4,1,27
4,1959,5,1,-1
5,1959,6,1,-1


Seuraavassa esimerkissä luetaan Apachen access.log-tiedostoa DataFrameen.  
![access.logia](http://student.labranet.jamk.fi/~varpe/datananalk2019/kerta3/accesslog.png)  

Erottimena on välilyönti mutta välilyöntejä on myös joidenkin kenttien sisällä `"GET /administrator/ HTTP/1.1"`.
Tästä ongelmasta päästään parametrilla `quotechar = '"'`, jolloin kaikki "-merkkien sisällä olevat erottimet jätetään huomioimatta.`  

In [12]:
# ensimmäinen yritys
df = pd.read_csv('https://bit.ly/2FQ4LsG', sep = ' ', quotechar = '"')


df.head()

,109.169.248.247,-,-.1,[12/Dec/2015:18:25:11,+0100],GET /administrator/ HTTP/1.1,200,4263,-.2,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20100101 Firefox/34.0,-.3
0,109.169.248.247,-,-,[12/Dec/2015:18:25:11,+0100],POST /administrator/index.php HTTP/1.1,200,4494,http://almhuette-raith.at/administrator/,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,-
1,46.72.177.4,-,-,[12/Dec/2015:18:31:08,+0100],GET /administrator/ HTTP/1.1,200,4263,-,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,-
2,46.72.177.4,-,-,[12/Dec/2015:18:31:08,+0100],POST /administrator/index.php HTTP/1.1,200,4494,http://almhuette-raith.at/administrator/,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,-
3,83.167.113.100,-,-,[12/Dec/2015:18:31:25,+0100],GET /administrator/ HTTP/1.1,200,4263,-,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,-
4,83.167.113.100,-,-,[12/Dec/2015:18:31:25,+0100],POST /administrator/index.php HTTP/1.1,200,4494,http://almhuette-raith.at/administrator/,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...,-


Ensimmäisestä yrityksestä nähdään, että otsikkoriviä ei tiedostossa ole, joten tehdään sarakeotsikot itse.   
Lisäksi voidaan skipata jotain sarakkeita.

In [13]:
# toinen yritys
df = pd.read_csv('https://bit.ly/2FQ4LsG', 
                 sep = ' ', 
                 quotechar = '"',
                 usecols = [0,3,4,5,6,7,8,9],  # nyt annetaan sarakkeiden numerot, kun nimiä ei vielä ole
                 names = ['ip', 'date', 'timezone', 'request', 'status', 'size', 'referer', 'user_agent']
                )
 
df.head()

,ip,date,timezone,request,status,size,referer,user_agent
0,109.169.248.247,[12/Dec/2015:18:25:11,+0100],GET /administrator/ HTTP/1.1,200,4263,-,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...
1,109.169.248.247,[12/Dec/2015:18:25:11,+0100],POST /administrator/index.php HTTP/1.1,200,4494,http://almhuette-raith.at/administrator/,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...
2,46.72.177.4,[12/Dec/2015:18:31:08,+0100],GET /administrator/ HTTP/1.1,200,4263,-,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...
3,46.72.177.4,[12/Dec/2015:18:31:08,+0100],POST /administrator/index.php HTTP/1.1,200,4494,http://almhuette-raith.at/administrator/,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...
4,83.167.113.100,[12/Dec/2015:18:31:25,+0100],GET /administrator/ HTTP/1.1,200,4263,-,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...


Myöhemmin opetellaan ajan parsiminen pandasin ymmärtämiksi kellonajoiksi, tässä vaiheessa voidaan tyytyä [-,merkin poistamiseen `converters`-parametrin avulla. Tälle annetaan sanakirjassa (dict) sarakeotsikot ja niille sarakkeille parsittaessa tehtävät funktiot:

In [1]:
# määritellään funktiot

def poistaalusta(x):
    return x[1:]

def poistalopusta(x):
    return x[:-1]


df = pd.read_csv('https://bit.ly/2FQ4LsG', 
                 sep = ' ', 
                 quotechar = '"',
                 usecols = [0,3,4,5,6,7,8,9],
                 names = ['ip', 'date', 'timezone', 'request', 'status', 'size', 'referer', 'user_agent'],
                 converters = {'date' : poistaalusta, 'timezone' : poistalopusta}
                )
df.head()

NameError: name 'pd' is not defined

Lyhyitä "yhden lauseen mittaisia" funktioita voidaan Pythonissa määritellä myös lambda-funktioina yhdelle riville:

In [15]:
tuplaa = lambda x : 2*x

summaa = lambda x,y : x+y

print(tuplaa(6))

print(summaa(3,4))

12
7


In [16]:
# määritellään funktiot

poistaalusta = lambda x : x[1:]

poistalopusta = lambda x : x[:-1]   

df = pd.read_csv('https://bit.ly/2FQ4LsG', 
                 sep = ' ', 
                 quotechar = '"',
                 usecols = [0,3,4,5,6,7,8,9],
                 names = ['ip', 'date', 'timezone', 'request', 'status', 'size', 'referer', 'user_agent'],
                 converters = {'date' : poistaalusta, 'timezone' : poistalopusta}
                )
df.head()

,ip,date,timezone,request,status,size,referer,user_agent
0,109.169.248.247,12/Dec/2015:18:25:11,+0100,GET /administrator/ HTTP/1.1,200,4263,-,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...
1,109.169.248.247,12/Dec/2015:18:25:11,+0100,POST /administrator/index.php HTTP/1.1,200,4494,http://almhuette-raith.at/administrator/,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...
2,46.72.177.4,12/Dec/2015:18:31:08,+0100,GET /administrator/ HTTP/1.1,200,4263,-,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...
3,46.72.177.4,12/Dec/2015:18:31:08,+0100,POST /administrator/index.php HTTP/1.1,200,4494,http://almhuette-raith.at/administrator/,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...
4,83.167.113.100,12/Dec/2015:18:31:25,+0100,GET /administrator/ HTTP/1.1,200,4263,-,Mozilla/5.0 (Windows NT 6.0; rv:34.0) Gecko/20...


lambda-funktiot voi määritellä myös suoraan `converters`-parametrin sisälle:

In [17]:
df = pd.read_csv('https://bit.ly/2FQ4LsG', 
                 sep = ' ', 
                 quotechar = '"',
                 usecols = [0,3,4,5,6,7,8,9],
                 names = ['ip', 'date', 'timezone', 'request', 'status', 'size', 'referer', 'user_agent'],
                 converters = {'date' : (lambda x : x[1:]), 'timezone' : (lambda x : x[:-1])}
                )


Muita parametreja `read_csv` -funktiolle on mm.  
* `skiprows`  skippaa tiedoston alusta annetun määrän rivejä, tai jos parametrin arvona annetaan lista, skippaa kyseiset rivinumerot
* `na_values`  tällä annetaan lista arvoista, jotka tulkitaan puuttuviksi arvoiksi.
* `skipinitialspace` skippaa erotinmerkin jälkeiset välilyönnit
* `comment`  määrittää minkä merkin jälkinen loppurivi jätetään huomioimatta ("kommenttimerkki")
* `error_bad_lines` oletuksena virhelliset rivit (esim liian monta kenttää) aiheuttavat virheilmoituksen ja parsiminen loppuu  siihen. Laittamalla `error_bad_lines = False` tällaiset rivit jätetään vain väliin

Alla luetaan tiedosto  
`rowid,lat,lon,type       
rowid1,62.21,31.21,wgs84      # first value... 
rowid2,62.21,31.21            # if format...
rowid3,62.24,33.21            # for decimal...
rowid4,6345147,3378945,ykj, 89
rowid5,6345,3378              # for ykj...`

In [18]:
df = pd.read_csv('https://bit.ly/2MrdfaU', comment = '#', error_bad_lines=False, index_col = 0)
df

b'Skipping line 5: expected 4 fields, saw 5\n'


,lat,lon,type
rowid,,,
rowid1,62.21,31.21,wgs84
rowid2,62.21,31.21,NaN
rowid3,62.24,33.21,NaN
rowid5,6345.00,3378.00,NaN


Tiedosto voi olla myös pakattu, pandas osaa sen purkaa tunnistaen pakkausformaatin tiedostopäätteestä (.gz .bz2 .zip tai .xz), käytetylle pakkaukselle on myös parametri `compression`.



<a id="2"> </a>
### Muita lukufunktioita
* `read_table` toimii kuten `read_csv` mutta oletuserottimena on \t (sarkain)
* `read_clipboard` lukee datan tiedoston sijasta leikepöydältä
* `read_excel` lukee Excel-tiedostosta yhden laskentataulukon (välilehden), jonka nimi annetaan toisena parametrina `pd.read_excel('examples/esim1.xlsx', 'Sheet1')`
* `read_html` lukee html-sivulta kaikki taulukot ja palauttaa listan DataFrame-objekteista
* `read_json`  lukee json-tiedostoa, usein json-tiedostot sisältävät paljon sisäkkäistä dataa, joka voi olla vaikea muotoilla DataFrameen
* `read_sql` lukee tietokantakyselyn tuloksen tai tietokantataulun DataFrameen 


In [24]:
url = 'https://online.tulospalvelu.fi/tulokset/fi/2018_smsprint/h21/smart/2/'

df = pd.read_html(url, header = 0)

df[0].head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Lähtöaika,"Väliaika0,3","Väliaika0,5","Väliaika2,9 km","Väliaika3,4 km","Maali3,6 km",Unnamed: 10
0,1.0,Olav Lundanes,Paimion Rasti,NOR,14:22:00,1:33 (1),2:00 (1),12:12 (1),13:32 (1),14:08,NaN
1,2.0,Severi Kymäläinen,Tampereen Pyrintö,FIN,14:32:00,1:37 (8),2:04 (3),12:15 (2),13:38 (2),14:17,NaN
2,3.0,Frederic Tranchand,Paimion Rasti,FRA,14:27:00,1:43 (22),2:09 (13),12:26 (5),13:45 (3),14:20,NaN
3,4.0,Yannick (U) Michiels,Turun Metsänkävijät,BEL,14:36:00,1:46 (31),2:13 (27),12:16 (3),13:46 (4),14:25,NaN
4,5.0,Joni Hirvikallio,Koovee,FIN,14:30:00,1:37 (8),2:05 (6),12:23 (4),14:01 (5),14:39,NaN


<a id="3"> </a>
## DataFramen kirjoittaminen tiedostoon

Vastaavasti DataFramen sisällön voi kirjoittaa tiedostoon, `to_csv` on funktio tähän tarkoitukseen.

Alla oleva esimerkki kirjoittaa  
`|Jukurit|Jyp|Sport
2014||5|
2015||3|14.0
2016||4|10.0
2017|11.0|3|14.0
2018|13.0|5|15.0`

Puuttuvat arvot jää oletuksena tyhjiksi, `na_rep='puuttuu'` -parametrilla niiden tilalle kirjoitettaisiin "puuttuu"

`to_html` -funktio kirjoittaa

```
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Jukurit</th>
      <th>Jyp</th>
      <th>Sport</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2014</th>
      <td>NaN</td>
      <td>5</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>2015</th>
      <td>NaN</td>
      <td>3</td>
      <td>14.0</td>
    </tr>
    <tr>
      <th>2016</th>
      <td>NaN</td>
      <td>4</td>
      <td>10.0</td>
    </tr>
    <tr>
      <th>2017</th>
      <td>11.0</td>
      <td>3</td>
      <td>14.0</td>
    </tr>
    <tr>
      <th>2018</th>
      <td>13.0</td>
      <td>5</td>
      <td>15.0</td>
    </tr>
  </tbody>
</table>
```

In [21]:
df = pd.DataFrame({'Jyp': {2014: 5, 2015: 3,  2016: 4,  2017: 3 , 2018: 5},
                   'Sport' : {2015: 14, 2016: 10, 2017: 14, 2018: 15 },
                    'Jukurit' : {2017: 11, 2018: 13}   
                  })

df.to_csv('esim1.txt', sep='|')

df.to_html('esim1.html')


<a id="4"> </a>
## Datan siivoaminen
<a id="5"> </a>
### Puuttuvien arvojen käsittely
Usein luettu data on epätäydellistä: siellä voi olla puuttuvia arvoja, duplikaatteja tms. 

pandas pärjää yleisesti ottaen hyvin puuttuvien arvojen kanssa, esim. keskiarvo, keskihajonta yms.- funktiot jättävät puuttuvat arvot huomioimatta. 

Puuttuvia arvoja sisältävät rivit (tai sarakkeet) voidaan kuitenkin haluta poistaa DataFramesta. Puuttuvat arvot voi listata `isnull`-metodin avulla (vastaavasti `notnull`-metodi palauttaa Truen niillä kohdilla jossa ei puutu arvo)

Poistamista varten DataFramella on metodi `dropna` joka oletuksena poistaa kaikki rivit joissa yksikin arvo on NaN.  
Parametrilla `how='all'` poistetaan vain ne rivit, joissa kaikki arvot puuttuvat,  
parametrilla `thresh` voidaan määrittää "kynnys" (thresold) kuinka monta arvoa riviltä pitää puuttua, jotta se poistetaan.  
Ja parametrilla `axis=1` tutkitaan/poistetaan rivien sijasta sarakkeita.


In [22]:
df = pd.DataFrame({'Jyp': {2014: 5, 2015: 3,  2016: 4,  2017: 3 , 2018: 5},
                   'Sport' : {2015: 14, 2016: 10, 2017: 14, 2018: 15 },
                    'Jukurit' : {2017: 11, 2018: 13}   
                  })

#print(df)

#print(20*'-')

print(df[df['Jukurit'].isnull()])  # suodatetaan df['Jukurit'].isnull() :n antaman Boolean-Seriesin avulla

print('\n------------------\n')

df2 = df.dropna(thresh=2)  # poistetaan ne rivit joissa vähintään 2 puuttuvaa arvoa

print(df2)


      Jyp  Sport  Jukurit
2014    5    NaN      NaN
2015    3   14.0      NaN
2016    4   10.0      NaN
2017    3   14.0     11.0
2018    5   15.0     13.0
--------------------
      Jyp  Sport  Jukurit
2014    5    NaN      NaN
2015    3   14.0      NaN
2016    4   10.0      NaN

------------------

      Jyp  Sport  Jukurit
2015    3   14.0      NaN
2016    4   10.0      NaN
2017    3   14.0     11.0
2018    5   15.0     13.0


Aukot eli puuttuvat arvot voidaan myös täyttää halutuilla arvoilla, ja tähän käytetään `fillna`-metodia.

In [23]:
df = pd.DataFrame({'Jyp': {2014: 5, 2015: 3,  2016: 4,  2017: 3 , 2018: 5},
                   'Sport' : {2015: 14, 2016: 10, 2017: 14, 2018: 15 },
                    'Jukurit' : {2017: 11, 2018: 13}   
                  })

print(df)

print('\n------------------\n')

df2 = df.fillna('mestis')

print(df2)

      Jyp  Sport  Jukurit
2014    5    NaN      NaN
2015    3   14.0      NaN
2016    4   10.0      NaN
2017    3   14.0     11.0
2018    5   15.0     13.0

------------------

      Jyp   Sport Jukurit
2014    5  mestis  mestis
2015    3      14  mestis
2016    4      10  mestis
2017    3      14      11
2018    5      15      13


In [27]:
df = pd.DataFrame({'Jyp': {2014: 5, 2015: 3,  2016: 4,  2017: 3 , 2018: 5},
                   'Blues' : {2014: 6, 2015: 5, 2016: 15},
                    'Jukurit' : {2017: 11, 2018: 13}   
                  })

print(df)

print('\n------------------\n')

df2 = df.fillna({'Jukurit' : 'mestis', 'Blues': 'konkurssi'})

print(df2)

      Jyp  Blues  Jukurit
2014    5    6.0      NaN
2015    3    5.0      NaN
2016    4   15.0      NaN
2017    3    NaN     11.0
2018    5    NaN     13.0

------------------

      Jyp      Blues Jukurit
2014    5          6  mestis
2015    3          5  mestis
2016    4         15  mestis
2017    3  konkurssi      11
2018    5  konkurssi      13


`fillna`-metodin `method='ffill'` -parametri saa pandasin täyttämään puuttuvat arvot sarakkeen edellisellä validilla arvolla, ja `method='bfill'`seuraavalla validilla arvolla.

<a id="6"> </a>
### Duplikaattien poistaminen
`drop_duplicates`-metodi poistaa täsmälleen samat arvot sisältävät rivit. Oletuksena se jättää ensimmäisen rivin jäljelle, parametrilla `keep='last'`viimeisen.

<a id="6b"> </a>
### replace
DataFramen `replace`-metodilla voidaan korvata arvoja. Sekä korvattaviin että korvaaviin arvoihin voidaan antaa yksittäisiä arvoja tai listaa, tai ainoana parametrina sanakirjan.  Tässäkin `inplace = True` tekee korvauksen alkuperäiseen DataFrameen, muuten uuteen.

In [24]:
df = pd.read_csv('https://bit.ly/2CFermr', sep=';', decimal=',', usecols=[0,1,2,5,6,7,8])

print(df)

print('\n-----------\n')

df['Kk'] = df['Kk'].replace(1, 'tammikuu')  # yksittäiset arvot

print(df)

print('\n-----------\n')

df['Kk'] = df['Kk'].replace('tammikuu', 1)

df['Kk'].replace([1, 2], ['tammikuu', 'helmikuu'], inplace=True)  # samanmittaiset listat

print(df)


print('\n-----------\n')



df['Kk'].replace({3: 'maalis', 4: 'huhti'}, inplace=True)   # sanakirja arvopareista korvattava:korvaava

print(df)


   Vuosi  Kk  Pv  Sademäärä  Lumensyvyys  Ylin  Alin
0   1959   1   1        1.0           34  -3.6  -9.8
1   1959   2   1       -1.0           63  -6.1 -13.1
2   1959   3   1       -1.0           55   8.2  -1.5
3   1959   4   1        7.8           27   5.0 -10.5
4   1959   5   1       -1.0           -1  17.9  -0.9
5   1959   6   1        3.6           -1  14.0   1.2

-----------

   Vuosi        Kk  Pv  Sademäärä  Lumensyvyys  Ylin  Alin
0   1959  tammikuu   1        1.0           34  -3.6  -9.8
1   1959         2   1       -1.0           63  -6.1 -13.1
2   1959         3   1       -1.0           55   8.2  -1.5
3   1959         4   1        7.8           27   5.0 -10.5
4   1959         5   1       -1.0           -1  17.9  -0.9
5   1959         6   1        3.6           -1  14.0   1.2

-----------

   Vuosi        Kk  Pv  Sademäärä  Lumensyvyys  Ylin  Alin
0   1959  tammikuu   1        1.0           34  -3.6  -9.8
1   1959  helmikuu   1       -1.0           63  -6.1 -13.1
2   1959   

<a id="10"> </a>
### apply, applymap, map
Serieksen `map`-metodilla voidaan tehdä muunnoksia tai funktioita alkioittain. Tälle voidaan antaa parametrina sanakirja tai Series, joiden perusteella määräytyvät tulevat arvot. Parametrina voi olla myös funktio, joka suoritetaan jokaiselle alkiolle.

In [25]:
df = pd.read_csv('https://bit.ly/2CFermr', sep=';', decimal=',', usecols=[0,1,2,5,6,7,8])

print(df)

print('\n-----------\n')

vuodenajat = pd.Series(['talvi','talvi','talvi','kevät','kevät','kesä','kesä','kesä','syksy','syksy','talvi','talvi'],
                       index=[1,2,3,4,5,6,7,8,9,10,11,12])
print(vuodenajat)

print('\n-----------\n')

df['vuodenaika'] = df['Kk'].map(vuodenajat)   # parametrina annettavan Serieksen indeksin mukaiset arvot korvataan Serieksen vastaavilla arvoilla

# parametrina voisi olla myös sanakirja {1: 'talvi', 2: 'talvi', 3: 'talvi', jne}

print(df)

print('\n-----------\n')


   Vuosi  Kk  Pv  Sademäärä  Lumensyvyys  Ylin  Alin
0   1959   1   1        1.0           34  -3.6  -9.8
1   1959   2   1       -1.0           63  -6.1 -13.1
2   1959   3   1       -1.0           55   8.2  -1.5
3   1959   4   1        7.8           27   5.0 -10.5
4   1959   5   1       -1.0           -1  17.9  -0.9
5   1959   6   1        3.6           -1  14.0   1.2

-----------

1     talvi
2     talvi
3     talvi
4     kevät
5     kevät
6      kesä
7      kesä
8      kesä
9     syksy
10    syksy
11    talvi
12    talvi
dtype: object

-----------

   Vuosi  Kk  Pv  Sademäärä  Lumensyvyys  Ylin  Alin vuodenaika
0   1959   1   1        1.0           34  -3.6  -9.8      talvi
1   1959   2   1       -1.0           63  -6.1 -13.1      talvi
2   1959   3   1       -1.0           55   8.2  -1.5      talvi
3   1959   4   1        7.8           27   5.0 -10.5      kevät
4   1959   5   1       -1.0           -1  17.9  -0.9      kevät
5   1959   6   1        3.6           -1  14.0   1.2       

In [27]:
df = pd.read_csv('https://bit.ly/2CFermr', sep=';', decimal=',', usecols=[0,1,2,5,6,7,8])

print(df)

# tässä annetaan parametrina funktio

df['sataa'] = df['Sademäärä'].map(lambda x: 'sataa' if x>0 else 'poutaa')  # tämän voisi toki tehdä monella helpommallakin tavalla
 
print('\n-----------\n')

print(df)

   Vuosi  Kk  Pv  Sademäärä  Lumensyvyys  Ylin  Alin
0   1959   1   1        1.0           34  -3.6  -9.8
1   1959   2   1       -1.0           63  -6.1 -13.1
2   1959   3   1       -1.0           55   8.2  -1.5
3   1959   4   1        7.8           27   5.0 -10.5
4   1959   5   1       -1.0           -1  17.9  -0.9
5   1959   6   1        3.6           -1  14.0   1.2

-----------

   Vuosi  Kk  Pv  Sademäärä  Lumensyvyys  Ylin  Alin   sataa
0   1959   1   1        1.0           34  -3.6  -9.8   sataa
1   1959   2   1       -1.0           63  -6.1 -13.1  poutaa
2   1959   3   1       -1.0           55   8.2  -1.5  poutaa
3   1959   4   1        7.8           27   5.0 -10.5   sataa
4   1959   5   1       -1.0           -1  17.9  -0.9  poutaa
5   1959   6   1        3.6           -1  14.0   1.2   sataa


DataFramen `applymap` -metodilla parametrina annettu funktio tehdään DataFramen jokaiselle alkioille. Alla tulostetaan arvot kahdella desimaalilla (muutetaan desimaaliluvut merkkijonoiksi)

In [31]:
import numpy as np

df = pd.DataFrame(np.random.randn(4, 3), columns=['a','b','c'], index=['JKL', 'HKI', 'TKU', 'TRE'])
print(df)

print('\n-----------\n')

def f(x):
    return "{:.2f}".format(x) 

print(df.applymap(f))

# lyhemmin

print(df.applymap(lambda x: "{:.2f}".format(x)))

# vieläkin lyhemmin

print(df.applymap("{:.2f}".format))

            a         b         c
JKL -1.113600 -0.384849  0.860849
HKI  1.011362 -1.982115  0.311728
TKU -0.999586 -0.620429  0.065679
TRE  1.180198 -0.595902  0.467932

-----------

         a      b     c
JKL  -1.11  -0.38  0.86
HKI   1.01  -1.98  0.31
TKU  -1.00  -0.62  0.07
TRE   1.18  -0.60  0.47
         a      b     c
JKL  -1.11  -0.38  0.86
HKI   1.01  -1.98  0.31
TKU  -1.00  -0.62  0.07
TRE   1.18  -0.60  0.47
         a      b     c
JKL  -1.11  -0.38  0.86
HKI   1.01  -1.98  0.31
TKU  -1.00  -0.62  0.07
TRE   1.18  -0.60  0.47



`apply`-metodi on määritelty sekä Seriekselle että DataFramelle ja sille annetaan parametrina funktio.  
Jos sitä kutsutaan DataFramella ja parametrina annetaan joku **koostefunktio** (sum, min, max jne), se laskee riveittäin tai sarakkeittain (`axis=1`) koostefunktion arvon. Jos taas annetaan funktio, joka ottaa yksittäisen argumentin, se toimii kuiten `applymap`

In [39]:
import numpy as np

df = pd.DataFrame(np.random.randn(4, 3), columns=['a','b','c'], index=['JKL', 'HKI', 'TKU', 'TRE'])
print(df)

print('\n-----------\n')

print(df.apply(lambda x: x.max()-x.min())) # voisi olla erikseen määritelty funktio def x(): ja kutsua apply(f)

print('\n-----------\n')

print(df.apply(lambda x: x.max()-x.min(), axis=1))  

print('\n-----------\n')

df['ero'] = df.apply(lambda x: x.max()-x.min(), axis=1)

print(df)


print('\n-----------\n')

print(df.apply(np.sqrt))    # tässä annetaan funktio, joka ottaa vain yhden luvun parametriksi



print('\n-----------\n')

def montakopositiivista(x):
    return x[x>0].count()


print(df.apply(montakopositiivista, axis=1))

            a         b         c
JKL  1.392332  0.904227  0.594571
HKI -0.689318  1.033189  0.111155
TKU -1.892242  0.880859  1.457926
TRE  1.145855 -0.267519 -0.371053

-----------

a    3.284575
b    1.300708
c    1.828979
dtype: float64

-----------

JKL    0.797761
HKI    1.722507
TKU    3.350168
TRE    1.516908
dtype: float64

-----------

            a         b         c       ero
JKL  1.392332  0.904227  0.594571  0.797761
HKI -0.689318  1.033189  0.111155  1.722507
TKU -1.892242  0.880859  1.457926  3.350168
TRE  1.145855 -0.267519 -0.371053  1.516908

-----------

            a         b         c       ero
JKL  1.179971  0.950909  0.771085  0.893175
HKI       NaN  1.016459  0.333398  1.312443
TKU       NaN  0.938541  1.207446  1.830346
TRE  1.070446       NaN       NaN  1.231628

-----------

JKL    4
HKI    3
TKU    3
TRE    2
dtype: int64


Vähän harvinaisempaa `apply`-funktion käyttöä:

In [32]:
df = pd.DataFrame(np.random.randn(4, 3), columns=['a','b','c'], index=['JKL', 'HKI', 'TKU', 'TRE'])
print(df)

print('\n-----------\n')

def tulosta(x):
    print(x.name + ' b = ' +  str(x['b']))


df.apply(tulosta, axis=1)

            a         b         c
JKL  1.050310 -1.294587 -0.340096
HKI -1.391876  1.378915  1.381623
TKU  0.136288  2.279925 -0.089195
TRE  0.958078 -1.028772  1.005734

-----------

JKL b = -1.294586664172514
HKI b = 1.3789153574385222
TKU b = 2.2799252599235467
TRE b = -1.0287720209616507


JKL    None
HKI    None
TKU    None
TRE    None
dtype: object

DataFramella on myös suoraan monia tilastometodeja (sum, mean jne), jolloin apply-metodin käyttö ei ole tarpeen

In [34]:
df = pd.DataFrame(np.random.randn(4, 3), columns=['a','b','c'], index=['JKL', 'HKI', 'TKU', 'TRE'])
print(df)

print('\n-----------\n')

print(df.apply(lambda x: x.sum())) 

print('\n-----------\n')

print(df.sum())   # sama asia

print('\n-----------\n')

print(df.apply(lambda x: x.sum(), axis=1)) 

print('\n-----------\n')

print(df.sum(axis=1))   # sama asia


            a         b         c
JKL -1.023474 -1.940957 -0.346192
HKI -0.300295  0.805941 -0.868341
TKU -0.649885 -0.443514 -0.117692
TRE -0.054942  0.415143  0.028638

-----------

a   -2.028595
b   -1.163387
c   -1.303588
dtype: float64

-----------

a   -2.028595
b   -1.163387
c   -1.303588
dtype: float64

-----------

JKL   -3.310623
HKI   -0.362695
TKU   -1.211091
TRE    0.388839
dtype: float64

-----------

JKL   -3.310623
HKI   -0.362695
TKU   -1.211091
TRE    0.388839
dtype: float64


Kootusti map, applymap ja apply-metodien erot:

  .|map|applymap|apply
--|-----|--------|-------
mille määritelty? |Series|DataFrame|Series ja DataFrame
parametri?|sanakirja, Series tai funktio|funktio|funktio
alkioittain?|kyllä|kyllä|kyllä
koostamiseen?|ei|ei|kyllä
käyttötarkoitus?|muunnokset, mappaus|muunnokset|monimutkaisemmat toiminnot
palauttaa?|Series|DataFrame|arvo, Series tai DataFrame

<a id="8"> </a>
### Kategorisointi
Dataa ryhmittää kategorioihin myös lokeroiden (välien) avulla. Esim. ikä-sarakkeesta voitaisiin generoida uusi sarake ikäryhmä, jonka avulla dataa voidaan analysoida eteenpäin (esim laskea keskiarvoja ikäryhmittäin).

Lokeroiden rajat määritellään listana, ja lokerointi tehdään pandasin `cut`-funktiolla. Oletuksena alaraja ei kuulu luokkaan mukaan, mutta `cut`-funktion parametrilla `right=False` tämä voidaan kääntää toisin päin.


In [37]:
df = pd.DataFrame({'Tappara' :{2001: 2, 2002: 2, 2003: 1, 2004: 9, 2005: 8, 2006: 5, 2007: 5, 2008: 3, 2009: 13, 2010: 7, 2011: 11, 2012: 12, 2013: 2, 2014: 2, 2015: 2, 2016: 1, 2017: 1, 2018: 2},
                  'Kärpät' :{2001: 4, 2002: 6, 2003: 2, 2004: 1, 2005: 1, 2006: 3, 2007: 1, 2008: 1, 2009: 2, 2010: 8, 2011: 10, 2012: 8, 2013: 9, 2014: 1, 2015: 1, 2016: 3, 2017: 10, 2018: 1},
                   'JYP' :{2001: 12, 2002: 9, 2003: 6, 2004: 10, 2005: 9, 2006: 9, 2007: 12, 2008: 5, 2009: 1, 2010: 3, 2011: 4, 2012: 1, 2013: 3, 2014: 5, 2015: 3, 2016: 4, 2017: 3, 2018: 5},
                   'KalPa' :{2006: 14, 2007: 14, 2008: 13, 2009: 3, 2010: 4, 2011: 6, 2012: 5, 2013: 6, 2014: 14, 2015: 6, 2016: 9, 2017: 2, 2018: 6}})
print(df)

print('\n-----------\n')
                  
lokerot = [0,3,8,10,15]

print(pd.cut(df['JYP'], lokerot))


      Tappara  Kärpät  JYP  KalPa
2001        2       4   12    NaN
2002        2       6    9    NaN
2003        1       2    6    NaN
2004        9       1   10    NaN
2005        8       1    9    NaN
2006        5       3    9   14.0
2007        5       1   12   14.0
2008        3       1    5   13.0
2009       13       2    1    3.0
2010        7       8    3    4.0
2011       11      10    4    6.0
2012       12       8    1    5.0
2013        2       9    3    6.0
2014        2       1    5   14.0
2015        2       1    3    6.0
2016        1       3    4    9.0
2017        1      10    3    2.0
2018        2       1    5    6.0

-----------

2001    (10, 15]
2002     (8, 10]
2003      (3, 8]
2004     (8, 10]
2005     (8, 10]
2006     (8, 10]
2007    (10, 15]
2008      (3, 8]
2009      (0, 3]
2010      (0, 3]
2011      (3, 8]
2012      (0, 3]
2013      (0, 3]
2014      (3, 8]
2015      (0, 3]
2016      (3, 8]
2017      (0, 3]
2018      (3, 8]
Name: JYP, dtype: category
Categor

Lokeroille, eli luokille voi antaa myös nimet (labels)

In [38]:
df = pd.DataFrame({'Tappara' :{2001: 2, 2002: 2, 2003: 1, 2004: 9, 2005: 8, 2006: 5, 2007: 5, 2008: 3, 2009: 13, 2010: 7, 2011: 11, 2012: 12, 2013: 2, 2014: 2, 2015: 2, 2016: 1, 2017: 1, 2018: 2},
                  'Kärpät' :{2001: 4, 2002: 6, 2003: 2, 2004: 1, 2005: 1, 2006: 3, 2007: 1, 2008: 1, 2009: 2, 2010: 8, 2011: 10, 2012: 8, 2013: 9, 2014: 1, 2015: 1, 2016: 3, 2017: 10, 2018: 1},
                   'JYP' :{2001: 12, 2002: 9, 2003: 6, 2004: 10, 2005: 9, 2006: 9, 2007: 12, 2008: 5, 2009: 1, 2010: 3, 2011: 4, 2012: 1, 2013: 3, 2014: 5, 2015: 3, 2016: 4, 2017: 3, 2018: 5},
                   'KalPa' :{2006: 14, 2007: 14, 2008: 13, 2009: 3, 2010: 4, 2011: 6, 2012: 5, 2013: 6, 2014: 14, 2015: 6, 2016: 9, 2017: 2, 2018: 6}})
print(df)

print('\n-----------\n')
                  
lokerot = [0,3,8,10,15]

luokat = ['mitali', 'play-offs', 'säälipleijarit', 'jäi runkosarjaan']

df['Jypin kausi'] =  pd.cut(df['JYP'], lokerot, labels=luokat)

print(df)


      Tappara  Kärpät  JYP  KalPa
2001        2       4   12    NaN
2002        2       6    9    NaN
2003        1       2    6    NaN
2004        9       1   10    NaN
2005        8       1    9    NaN
2006        5       3    9   14.0
2007        5       1   12   14.0
2008        3       1    5   13.0
2009       13       2    1    3.0
2010        7       8    3    4.0
2011       11      10    4    6.0
2012       12       8    1    5.0
2013        2       9    3    6.0
2014        2       1    5   14.0
2015        2       1    3    6.0
2016        1       3    4    9.0
2017        1      10    3    2.0
2018        2       1    5    6.0

-----------

      Tappara  Kärpät  JYP  KalPa       Jypin kausi
2001        2       4   12    NaN  jäi runkosarjaan
2002        2       6    9    NaN    säälipleijarit
2003        1       2    6    NaN         play-offs
2004        9       1   10    NaN    säälipleijarit
2005        8       1    9    NaN    säälipleijarit
2006        5       3    9  

Jos `cut`-funktiolle antaa luokkarajalistan sijasta kokonaisluvun *n*, pandas luo itse *n* yhtäsuurta luokkaa datan pienimmän ja suurimman arvon välille.

In [11]:
import numpy as np

data = np.random.rand(20)

print(data)

print('\n-----------\n')

print(pd.cut(data, 4, precision=2))

[ 0.38134537  0.91983637  0.7234436   0.12880192  0.80078357  0.56469369
  0.51317106  0.619319    0.19249768  0.06094079  0.221748    0.82255354
  0.09298374  0.49738777  0.37747169  0.47338012  0.94583261  0.52809823
  0.5009749   0.0428811 ]
[(0.27, 0.49], (0.72, 0.95], (0.72, 0.95], (0.042, 0.27], (0.72, 0.95], ..., (0.27, 0.49], (0.72, 0.95], (0.49, 0.72], (0.49, 0.72], (0.042, 0.27]]
Length: 20
Categories (4, interval[float64]): [(0.042, 0.27] < (0.27, 0.49] < (0.49, 0.72] < (0.72, 0.95]]


<a id="9"> </a>
### NumPy-funktiot DataFramelle
NumPyn matemaattiset funktiot jotka tehdään NumPy-taulukolle alkioittain toimivat myös Seriesille tai DataFramelle.

In [141]:
df = pd.DataFrame(np.random.randn(4, 3), columns=['a','b','c'], index=['JKL', 'HKI', 'TKU', 'TRE'])
print(df)

print('\n-----------\n')

print(np.abs(df))  #itseisarvo


            a         b         c
JKL  0.881685  0.374174  0.573517
HKI  0.222266  0.906210 -0.618325
TKU  0.782257 -0.174971 -0.518199
TRE -0.426967 -0.089095 -0.623538

-----------

            a         b         c
JKL  0.881685  0.374174  0.573517
HKI  0.222266  0.906210  0.618325
TKU  0.782257  0.174971  0.518199
TRE  0.426967  0.089095  0.623538


<a id="11"> </a>
### pandasin merkkijonofunktiot

pandasissa on myös omia merkkijonofunktioita, jotka tekevät operaatiot alkioittain kaikille Seriesin alkioille. 
Toinen vaihtoehto olisi käyttää `map`-metodia, ja siinä normaaleja Pythonin merkkijonofunktioita, mutta tällöin ongelma on että ne kaatuvat jos parametrina tulee NaN eli puuttuva arvo. pandasin merkkijonofunktiot selviävät NaN-arvoistakin.

pandasin merkkijonofunktioita kutsutaan .str.funktio

[pandasin merkkijonofunktioista](http://pandas.pydata.org/pandas-docs/stable/text.html#)

In [159]:
data = {'email': {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan}}

df = pd.DataFrame(data)

print(df)

print('\n--------------\n')



# df['gmail'] = df['email'].map(lambda x: x.find('gmail')>-1) 
# ei toimi, koska NaN-arvo antaa "'float' object has no attribute 'find'"

# sen sijaan pandasin merkkijonofunktio contains toimii:
df['gmail'] = df['email'].str.contains('gmail')

df['domain'] = df['email'].str.split('@').str.get(1).str.split('.').str.get(0)

df['ekat'] = df['email'].str[0:3]

print(df)


                 email
Dave   dave@google.com
Rob      rob@gmail.com
Steve  steve@gmail.com
Wes                NaN

--------------

                 email  gmail  domain ekat
Dave   dave@google.com  False  google  dav
Rob      rob@gmail.com   True   gmail  rob
Steve  steve@gmail.com   True   gmail  ste
Wes                NaN    NaN     NaN  NaN
